# Overview

This notebook is based on this [Prediction and Feature Store Online Serving](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/feature_store/mobile_gaming/mobile_gaming_feature_store.ipynb) notebook and [this blog post](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml)

### Dataset

The dataset is the public sample export data from an actual mobile game app called "Flood It!" (Android, iOS)

### Model

The model you use in this notebook is based on [this blog post](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml). The idea behind this model is that your company has extensive log data describing how your game users have interacted with the site. The raw data contains the following categories of information:

- identity - unique player identitity numbers
- demographic features - information about the player, such as the geographic region in which a player is located
- behavioral features - counts of the number of times a  player has triggered certain game events, such as reaching a new level
- churn propensity - this is the label or target feature, it provides an estimated probability that this player will churn, i.e. stop being an active player.

The blog article referenced above explains how to use BigQuery to store the raw data, pre-process the data for machine learning, and train the corresponding model. Because this notebook focuses on model monitoring, rather than training models, you're going to reuse a pre-trained version of this model, which has been exported to Cloud Storage, which is stored in `gs://mco-mm/churn`

# Basic set up

## Install packages

In [ ]:
! pip install google-cloud-aiplatform

## Set up project

In [ ]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
shell_output = ! gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = shell_output[0]
print("Project Number:", PROJECT_NUMBER)

In [ ]:
! gcloud config set project $PROJECT_ID

The feature is only available in `us-central1` now

In [ ]:
REGION = "us-central1"

## Authenticate your Google Cloud account

If you are using Vertex AI Workbench Notebooks, your environment is already authenticated. Skip this step.

If you are using Colab, run the cell below and follow the instructions when prompted to authenticate your account via oAuth.

Otherwise, follow these steps:

1. In the Cloud Console, go to the Create service account key page.

2. Click Create service account.

3. In the Service account name field, enter a name, and click Create.

4. In the Grant this service account access to project section, click the Role drop-down list and add the following roles:

- BigQuery Admin
- Storage Admin
- Storage Object Admin
- Vertex AI Administrator
- Vertex AI Feature Store Admin

5. Click Create. A JSON file that contains your key downloads to your local environment.

6. Enter the path to your service account key as the `GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

Create a bucket and copy the exported model to it

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "-aip-" + TIMESTAMP
    BUCKET_URI = f"gs://{BUCKET_NAME}"

Only if your bucket doesn't already exist: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

In [ ]:
! gsutil ls -al $BUCKET_URI

Copy the trained model to your bucket

In [ ]:
! gsutil cp -r gs://mco-mm/churn $BUCKET_URI

In [ ]:
ARTIFACT_URI = f"gs://{BUCKET_NAME}/churn"

### Create a service account

We need a service account for this new feature because the prediction workload's credential does not have access to Feature Store. Create one and grant the role `roles/aiplatform.serviceAgent` to it, which will grant it the access to most prediction's resources and Feature Store.

In [ ]:
SA_NAME = "prediction-and-fs-testing"
SA_DESCRIPTION = '"SA to test Prediction and Feature Store integration"'
DISPLAY_NAME = "prediction-and-fs-testing"

In [ ]:
!gcloud iam service-accounts create $SA_NAME \
    --description=$SA_DESCRIPTION \
    --display-name=$DISPLAY_NAME

In [ ]:
# SERVICE_ACCOUNT = "prediction-and-fs@bon-test-0.iam.gserviceaccount.com"
SERVICE_ACCOUNT = (
    f"{SA_NAME}@{PROJECT_ID}.iam.gserviceaccount.com"  # @param {type:"string"}
)

Grant the Service account the `Vertex AI Service Agent` role which has access to most Vertex AI resources including Feature Store.

In [ ]:
!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:$SERVICE_ACCOUNT \
    --role=roles/aiplatform.serviceAgent;

## Timestamp

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

# Create Feature Store

## Get data from Big Query

In [ ]:
BQ_DATASET = "Mobile_Gaming_FS_Pred"  # @param {type:"string"}
LOCATION = "US"

In [ ]:
!bq mk --location=$LOCATION --dataset $PROJECT_ID:$BQ_DATASET

In [ ]:
# General
import os
import sys

# Vertex AI and its Feature Store
from google.cloud import bigquery
from google.cloud.aiplatform import Featurestore

In [ ]:
# Data Engineering and Feature Engineering
LABEL_TABLE = f"label_table_{TIMESTAMP}".replace("-", "")
FEATURES_TABLE = f"wide_features_table_{TIMESTAMP}"  # @param {type:"string"}
FEATURESTORE_ID = "mobile_gaming4"  # @param {type:"string"}

# Vertex AI Feature store
ONLINE_STORE_NODES_COUNT = 5
DEMOGRAPHIC_ENTITY_ID = "demographic"
BEHAVIOR_ENTITY_ID = "behavior"
FEATURE_TIME = "timestamp"
ENTITY_ID_FIELD = "user_pseudo_id"
BQ_SOURCE_URI = f"bq://{PROJECT_ID}.{BQ_DATASET}.{FEATURES_TABLE}"

## Query data

In [ ]:
bq_client = bigquery.Client(project=PROJECT_ID, location=LOCATION)

This notebook is mainly to demo our new feature, so, I'll just query 1000 non-null rows

In [ ]:
features_sql_query = f"""
CREATE OR REPLACE TABLE
  `{PROJECT_ID}.{BQ_DATASET}.{FEATURES_TABLE}` AS
WITH
  # query to extract demographic data for each user ---------------------------------------------------------
  get_demographic_data AS (
  SELECT * EXCEPT (row_num)
  FROM (
    SELECT
      user_pseudo_id,
      geo.country as country,
      device.operating_system as operating_system,
      device.language as language,
      ROW_NUMBER() OVER (PARTITION BY user_pseudo_id ORDER BY event_timestamp DESC) AS row_num
    FROM `firebase-public-project.analytics_153293282.events_*`)
  WHERE row_num = 1),

  # query to extract behavioral data for each user ----------------------------------------------------------
  get_behavioral_data AS (
  SELECT
    event_timestamp,
    user_pseudo_id,
    SUM(IF(event_name = 'user_engagement', 1, 0)) OVER (PARTITION BY user_pseudo_id ORDER BY event_timestamp ASC RANGE BETWEEN 21600000000 PRECEDING
      AND CURRENT ROW ) AS cnt_user_engagement,
    SUM(IF(event_name = 'level_start_quickplay', 1, 0)) OVER (PARTITION BY user_pseudo_id ORDER BY event_timestamp ASC RANGE BETWEEN 21600000000 PRECEDING
      AND CURRENT ROW ) AS cnt_level_start_quickplay,
    SUM(IF(event_name = 'level_end_quickplay', 1, 0)) OVER (PARTITION BY user_pseudo_id ORDER BY event_timestamp ASC RANGE BETWEEN 21600000000 PRECEDING
      AND CURRENT ROW ) AS cnt_level_end_quickplay,
    SUM(IF(event_name = 'level_complete_quickplay', 1, 0)) OVER (PARTITION BY user_pseudo_id ORDER BY event_timestamp ASC RANGE BETWEEN 21600000000 PRECEDING
      AND CURRENT ROW ) AS cnt_level_complete_quickplay,
    SUM(IF(event_name = 'level_reset_quickplay', 1, 0)) OVER (PARTITION BY user_pseudo_id ORDER BY event_timestamp ASC RANGE BETWEEN 21600000000 PRECEDING
      AND CURRENT ROW ) AS cnt_level_reset_quickplay,
    SUM(IF(event_name = 'post_score', 1, 0)) OVER (PARTITION BY user_pseudo_id ORDER BY event_timestamp ASC RANGE BETWEEN 21600000000 PRECEDING
      AND CURRENT ROW ) AS cnt_post_score,
    SUM(IF(event_name = 'spend_virtual_currency', 1, 0)) OVER (PARTITION BY user_pseudo_id ORDER BY event_timestamp ASC RANGE BETWEEN 21600000000 PRECEDING
      AND CURRENT ROW ) AS cnt_spend_virtual_currency,
    SUM(IF(event_name = 'ad_reward', 1, 0)) OVER (PARTITION BY user_pseudo_id ORDER BY event_timestamp ASC RANGE BETWEEN 21600000000 PRECEDING
      AND CURRENT ROW ) AS cnt_ad_reward,
    SUM(IF(event_name = 'challenge_a_friend', 1, 0)) OVER (PARTITION BY user_pseudo_id ORDER BY event_timestamp ASC RANGE BETWEEN 21600000000 PRECEDING
      AND CURRENT ROW ) AS cnt_challenge_a_friend,
    SUM(IF(event_name = 'completed_5_levels', 1, 0)) OVER (PARTITION BY user_pseudo_id ORDER BY event_timestamp ASC RANGE BETWEEN 21600000000 PRECEDING
      AND CURRENT ROW ) AS cnt_completed_5_levels,
    SUM(IF(event_name = 'use_extra_steps', 1, 0)) OVER (PARTITION BY user_pseudo_id ORDER BY event_timestamp ASC RANGE BETWEEN 21600000000 PRECEDING
      AND CURRENT ROW ) AS cnt_use_extra_steps,
  FROM (
    SELECT
      e.*
    FROM
      `firebase-public-project.analytics_153293282.events_*` AS e
    )
  ),

  # query to extract the first and last time user play the game --------------------------------------------
  first_last_touch AS (
    SELECT
      user_pseudo_id,
      MIN(event_timestamp) AS user_first_engagement,
      MAX(event_timestamp) AS user_last_engagement
    FROM
      `firebase-public-project.analytics_153293282.events_*`
    WHERE event_name="user_engagement"
    GROUP BY
      user_pseudo_id

  )

SELECT full_data.*
FROM (
SELECT
    TIMESTAMP_ADD(PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%S', FORMAT_TIMESTAMP('%Y-%m-%d %H:%M:%S', TIMESTAMP_MICROS(beh.event_timestamp))), INTERVAL 1351 DAY) AS timestamp,
    dem.*,
    CAST(IFNULL(beh.cnt_user_engagement, 0) AS FLOAT64)  AS cnt_user_engagement,
    CAST(IFNULL(beh.cnt_level_start_quickplay, 0) AS FLOAT64) AS cnt_level_start_quickplay,
    CAST(IFNULL(beh.cnt_level_end_quickplay, 0) AS FLOAT64) AS cnt_level_end_quickplay,
    CAST(IFNULL(beh.cnt_level_complete_quickplay, 0) AS FLOAT64) AS cnt_level_complete_quickplay,
    CAST(IFNULL(beh.cnt_level_reset_quickplay, 0) AS FLOAT64) AS cnt_level_reset_quickplay,
    CAST(IFNULL(beh.cnt_post_score, 0) AS FLOAT64) AS cnt_post_score,
    CAST(IFNULL(beh.cnt_spend_virtual_currency, 0) AS FLOAT64) AS cnt_spend_virtual_currency,
    CAST(IFNULL(beh.cnt_ad_reward, 0) AS FLOAT64) AS cnt_ad_reward,
    CAST(IFNULL(beh.cnt_challenge_a_friend, 0) AS FLOAT64) AS cnt_challenge_a_friend,
    CAST(IFNULL(beh.cnt_completed_5_levels, 0) AS FLOAT64) AS cnt_completed_5_levels,
    CAST(IFNULL(beh.cnt_use_extra_steps, 0) AS FLOAT64) AS cnt_use_extra_steps,
    flt.user_first_engagement,
    flt.user_last_engagement,
    EXTRACT(MONTH from TIMESTAMP_MICROS(flt.user_first_engagement)) as month,
    EXTRACT(DAYOFYEAR from TIMESTAMP_MICROS(flt.user_first_engagement)) as julianday,
    EXTRACT(DAYOFWEEK from TIMESTAMP_MICROS(flt.user_first_engagement)) as dayofweek,
    
FROM
  get_demographic_data dem
LEFT OUTER JOIN 
  get_behavioral_data beh
ON
  dem.user_pseudo_id = beh.user_pseudo_id
LEFT OUTER JOIN
  first_last_touch flt
ON
  dem.user_pseudo_id = flt.user_pseudo_id) as full_data
# filter to only get rows with all non-null data
WHERE NOT REGEXP_CONTAINS(TO_JSON_STRING(full_data), r':null[,}}]')
# to save time for the demo, only get the first 1000 rows
LIMIT 1000
"""

In [ ]:
try:
    job = bq_client.query(features_sql_query)
    _ = job.result()
except RuntimeError as error:
    print(error)

## Create Feature Store

In [ ]:
try:
    mobile_gaming_feature_store = Featurestore.create(
        featurestore_id=FEATURESTORE_ID,
        online_store_fixed_node_count=ONLINE_STORE_NODES_COUNT,
        sync=True,
    )
except RuntimeError as error:
    print(error)
else:
    FEATURESTORE_RESOURCE_NAME = mobile_gaming_feature_store.resource_name
    print(f"Feature store created: {FEATURESTORE_RESOURCE_NAME}")

### Create Entities

In [ ]:
try:
    demographic_entity_type = mobile_gaming_feature_store.create_entity_type(
        entity_type_id=DEMOGRAPHIC_ENTITY_ID,
        description="User demographic Entity",
        sync=True,
    )
except RuntimeError as error:
    print(error)
else:
    DEMOGRAPHIC_ENTITY_RESOURCE_NAME = demographic_entity_type.resource_name
    print("Entity type name is", DEMOGRAPHIC_ENTITY_RESOURCE_NAME)

In [ ]:
try:
    behavior_entity_type = mobile_gaming_feature_store.create_entity_type(
        entity_type_id=BEHAVIOR_ENTITY_ID, description="User behavior Entity", sync=True
    )
except RuntimeError as error:
    print(error)
else:
    BEHAVIOR_ENTITY_RESOURCE_NAME = behavior_entity_type.resource_name
    print("Entity type name is", BEHAVIOR_ENTITY_RESOURCE_NAME)

### Create Features

#### Feature Config

In [ ]:
demographic_feature_configs = {
    "country": {
        "value_type": "STRING",
        "description": "The country of customer",
        "labels": {"status": "passed"},
    },
    "operating_system": {
        "value_type": "STRING",
        "description": "The operating system of device",
        "labels": {"status": "passed"},
    },
    "language": {
        "value_type": "STRING",
        "description": "The language of device",
        "labels": {"status": "passed"},
    },
    "user_pseudo_id": {
        "value_type": "STRING",
        "description": "User pseudo id",
        "labels": {"status": "passed"},
    },
}

behavior_feature_configs = {
    "cnt_user_engagement": {
        "value_type": "DOUBLE",
        "description": "A variable of user engagement level",
        "labels": {"status": "passed"},
    },
    "cnt_level_start_quickplay": {
        "value_type": "DOUBLE",
        "description": "A variable of user engagement with start level",
        "labels": {"status": "passed"},
    },
    "cnt_level_end_quickplay": {
        "value_type": "DOUBLE",
        "description": "A variable of user engagement with end level",
        "labels": {"status": "passed"},
    },
    "cnt_level_complete_quickplay": {
        "value_type": "DOUBLE",
        "description": "A variable of user engagement with complete status",
        "labels": {"status": "passed"},
    },
    "cnt_level_reset_quickplay": {
        "value_type": "DOUBLE",
        "description": "A variable of user engagement with reset status",
        "labels": {"status": "passed"},
    },
    "cnt_post_score": {
        "value_type": "DOUBLE",
        "description": "A variable of user score",
        "labels": {"status": "passed"},
    },
    "cnt_spend_virtual_currency": {
        "value_type": "DOUBLE",
        "description": "A variable of user virtual amount",
        "labels": {"status": "passed"},
    },
    "cnt_ad_reward": {
        "value_type": "DOUBLE",
        "description": "A variable of user reward",
        "labels": {"status": "passed"},
    },
    "cnt_challenge_a_friend": {
        "value_type": "DOUBLE",
        "description": "A variable of user challenges with friends",
        "labels": {"status": "passed"},
    },
    "cnt_completed_5_levels": {
        "value_type": "DOUBLE",
        "description": "A variable of user level 5 completed",
        "labels": {"status": "passed"},
    },
    "cnt_use_extra_steps": {
        "value_type": "DOUBLE",
        "description": "A variable of user extra steps",
        "labels": {"status": "passed"},
    },
    "month": {
        "value_type": "INT64",
        "description": "First touch month",
        "labels": {"status": "passed"},
    },
    "julianday": {
        "value_type": "INT64",
        "description": "First touch julian day",
        "labels": {"status": "passed"},
    },
    "dayofweek": {
        "value_type": "INT64",
        "description": "First touch day of week",
        "labels": {"status": "passed"},
    },
}

#### Create features using `batch_create_features` method

In [ ]:
try:
    demographic_entity_type.batch_create_features(
        feature_configs=demographic_feature_configs, sync=True
    )
except RuntimeError as error:
    print(error)
else:
    for feature in demographic_entity_type.list_features():
        print("")
        print(f"The resource name of {feature.name} feature is", feature.resource_name)

In [ ]:
try:
    behavior_entity_type.batch_create_features(
        feature_configs=behavior_feature_configs, sync=True
    )
except RuntimeError as error:
    print(error)
else:
    for feature in behavior_entity_type.list_features():
        print("")
        print(f"The resource name of {feature.name} feature is", feature.resource_name)

### Ingest features 

In [ ]:
DEMOGRAPHIC_FEATURES_IDS = [
    feature.name for feature in demographic_entity_type.list_features()
]

In [ ]:
try:
    demographic_entity_type.ingest_from_bq(
        feature_ids=DEMOGRAPHIC_FEATURES_IDS,
        feature_time=FEATURE_TIME,
        bq_source_uri=BQ_SOURCE_URI,
        entity_id_field=ENTITY_ID_FIELD,
        disable_online_serving=False,
        worker_count=10,
        sync=False,
    )
except RuntimeError as error:
    print(error)

In [ ]:
BEHAVIOR_FEATURES_IDS = [
    feature.name for feature in behavior_entity_type.list_features()
]

In [ ]:
try:
    behavior_entity_type.ingest_from_bq(
        feature_ids=BEHAVIOR_FEATURES_IDS,
        feature_time=FEATURE_TIME,
        bq_source_uri=BQ_SOURCE_URI,
        entity_id_field=ENTITY_ID_FIELD,
        disable_online_serving=False,
        worker_count=10,
        sync=False,
    )
except RuntimeError as error:
    print(error)

# Create Feature Fetch Config

In [ ]:
entity_config = {
    "demographic": demographic_feature_configs,
    "behavior": behavior_feature_configs,
}

In [ ]:
feature_config_string = """modelInputFormat: DICT
features:"""

for entity_id, config in entity_config.items():
    for feature_name in config.keys():
        feature_config_string += """
- valueKey: {FEATURE_NAME}
  featureSource:
    entityIdKey: {ENTITY_ID}
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/{ENTITY_ID}
    featureId: {FEATURE_NAME}""".format(
            FEATURE_NAME=feature_name,
            ENTITY_ID=entity_id,
            PROJECT_NUMBER=PROJECT_NUMBER,
            REGION=REGION,
            FEATURESTORE_ID=FEATURESTORE_ID,
        )

In [ ]:
print(feature_config_string)

In [ ]:
with open("prediction_featuresstore_fetch_config.yaml", "w") as f:
    f.write(feature_config_string)

In [ ]:
ARTIFACT_URI

In [ ]:
# Remove if the file already exists
!gsutil rm $ARTIFACT_URI/prediction_featuresstore_fetch_config.yaml

In [ ]:
!gsutil cp prediction_featuresstore_fetch_config.yaml $ARTIFACT_URI

# Integrate with Vertex Prediction

**NOTE:** The feature is only available in `autopush` now

In [ ]:
import google.cloud.aiplatform as aip

aip.constants.base.API_BASE_PATH = "autopush-aiplatform.sandbox.googleapis.com"
aip.constants.base.PREDICTION_API_BASE_PATH = (
    "autopush-prediction-aiplatform.sandbox.googleapis.com"
)

from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

## Upload Model

In [ ]:
DEPLOY_IMAGE = "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest"

In [ ]:
model = aip.Model.upload(
    display_name="mobile_gaming_featureStore_integration_" + TIMESTAMP,
    artifact_uri=ARTIFACT_URI,
    serving_container_image_uri=DEPLOY_IMAGE,
    sync=False,
)

model.wait()

## Deploy Model

In [ ]:
DEPLOYED_NAME = "mobile_gaming_featureStore_integration_" + TIMESTAMP

TRAFFIC_SPLIT = {"0": 100}

MACHINE_TYPE = "n1-standard-4"

MIN_NODES = 1
MAX_NODES = 1

endpoint = model.deploy(
    deployed_model_display_name=DEPLOYED_NAME,
    traffic_split=TRAFFIC_SPLIT,
    machine_type=MACHINE_TYPE,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    service_account=SERVICE_ACCOUNT,
)

## Predict

### Get Test Prediction Request

In [ ]:
MODEL_INPUT_QUERY = f"""
WITH user_latest_action AS (
  SELECT 
    user_pseudo_id,
    MAX(timestamp) AS timestamp
  FROM
    `{PROJECT_ID}.{BQ_DATASET}.{FEATURES_TABLE}` AS t
  GROUP BY user_pseudo_id
)
SELECT 
  country,
  cnt_spend_virtual_currency,
  language,
  cnt_completed_5_levels,
  user_latest_action.user_pseudo_id,
  cnt_level_reset_quickplay,
  dayofweek,
  cnt_ad_reward,
  cnt_challenge_a_friend,
  cnt_level_end_quickplay,
  cnt_level_complete_quickplay,
  julianday,
  cnt_post_score,
  cnt_level_start_quickplay,
  month,
  cnt_user_engagement,
  operating_system,
  cnt_use_extra_steps,
FROM
  user_latest_action 
LEFT JOIN
  `{PROJECT_ID}.{BQ_DATASET}.{FEATURES_TABLE}` AS t
ON
  user_latest_action.user_pseudo_id=t.user_pseudo_id AND
  user_latest_action.timestamp=t.timestamp
LIMIT 5
"""

In [ ]:
try:
    job = bq_client.query(MODEL_INPUT_QUERY)
    model_input_query_result = job.result()
except RuntimeError as error:
    print(error)

In [ ]:
DEFAULT_PRED_REQUEST = [dict(row) for row in model_input_query_result]
DEFAULT_PRED_REQUEST

In [ ]:
FS_PRED_REQUEST = [
    {
        "demographic": default_input["user_pseudo_id"],
        "behavior": default_input["user_pseudo_id"],
    }
    for default_input in DEFAULT_PRED_REQUEST
]
FS_PRED_REQUEST

In [ ]:
endpoint.predict([DEFAULT_PRED_REQUEST[0]])

In [ ]:
DEFAULT_RESPONSE = endpoint.predict(DEFAULT_PRED_REQUEST)

### Single instance

In [ ]:
endpoint.predict([FS_PRED_REQUEST[0]])

### Multiple instances

In [ ]:
FS_RESPONSE = endpoint.predict(FS_PRED_REQUEST)

### Compare response

In [ ]:
DEFAULT_RESPONSE.predictions == FS_RESPONSE.predictions

# Clean Up

In [ ]:
# delete feature store
mobile_gaming_feature_store.delete(sync=True, force=True)

In [ ]:
# delete Vertex AI resources
endpoint.undeploy_all()
endpoint.delete()
model.delete

In [ ]:
# Delete bucket
!gsutil -m rm -r $BUCKET_URI

In [ ]:
# Delete the BigQuery Dataset
!bq rm -r -f -d $PROJECT_ID:$BQ_DATASET